In [1]:
from smartsim import Experiment
from smartsim.database import SlurmOrchestrator
from smartsim.settings import SrunSettings
from smartsim import slurm

def launch_cluster_orc(experiment, port, alloc):
    """Just spin up a database cluster, check the status
       and tear it down"""

    db = SlurmOrchestrator(port=port,
                            db_nodes=1,
                            batch=False,
                            alloc=alloc,
                            interface="ib0")
    

    # generate directories for output files
    # pass in objects to make dirs for
    experiment.generate(db, overwrite=True)

    # start the database on interactive allocation
    experiment.start(db, block=True)

    return db

def create_uploader(experiment, alloc, nodes=1, tasks_per_node=1):

    srun = SrunSettings(exe="python",
                        exe_args="data_uploader.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_nodes(nodes)
    srun.set_tasks_per_node(tasks_per_node)

    uploader = experiment.create_ensemble("uploader", replicas=2, run_settings=srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    uploader.attach_generator_files(to_copy=["./torch/data_uploader.py"])
    experiment.generate(uploader, overwrite=True)
    return uploader


def create_trainer(experiment, alloc):

    srun = SrunSettings(exe="python",
                        exe_args="training_service.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_tasks(1)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./torch/training_service.py")
    experiment.generate(trainer, overwrite=True)
    return trainer


def create_trainer_hvd(experiment, alloc, nodes=1, tasks_per_node=1):

    srun = SrunSettings(exe="python",
                        exe_args="training_service_hvd.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_nodes(nodes)
    srun.set_tasks_per_node(tasks_per_node)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./torch/training_service_hvd.py")
    experiment.generate(trainer, overwrite=True)
    return trainer

In [2]:
alloc = slurm.get_allocation(nodes=4, time="03:00:00", options={"constraint": "V100", "partition": "spider"})

11:23:32 osprey.us.cray.com SmartSim[126059] INFO Allocation successful with Job ID: 528401


In [5]:
exp = Experiment("launch_streaming", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 8)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer(exp, alloc)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

11:32:28 osprey.us.cray.com SmartSim[126059] INFO Working in previously created experiment
11:33:01 osprey.us.cray.com SmartSim[126059] INFO Working in previously created experiment
11:33:08 osprey.us.cray.com SmartSim[126059] INFO Working in previously created experiment
11:33:16 osprey.us.cray.com SmartSim[126059] INFO uploader_0(528401.5): New
11:33:16 osprey.us.cray.com SmartSim[126059] INFO uploader_1(528401.6): New
11:33:16 osprey.us.cray.com SmartSim[126059] INFO trainer(528401.7): New
11:33:21 osprey.us.cray.com SmartSim[126059] INFO uploader_0(528401.5): Running
11:33:21 osprey.us.cray.com SmartSim[126059] INFO uploader_1(528401.6): Running
11:33:21 osprey.us.cray.com SmartSim[126059] INFO trainer(528401.7): Running
11:33:26 osprey.us.cray.com SmartSim[126059] INFO uploader_0(528401.5): Running
11:33:26 osprey.us.cray.com SmartSim[126059] INFO uploader_1(528401.6): Running
11:33:26 osprey.us.cray.com SmartSim[126059] INFO trainer(528401.7): Running
11:33:31 osprey.us.cray.com 

KeyboardInterrupt: 

In [6]:
exp.stop(db, uploader_model, trainer_model)

11:46:13 osprey.us.cray.com SmartSim[126059] INFO Stopping model trainer with job name trainer-CG4ZUZNVHESN
11:46:13 osprey.us.cray.com SmartSim[126059] INFO Stopping model uploader_0 with job name uploader_0-CG4ZUWNONLMY
11:46:14 osprey.us.cray.com SmartSim[126059] INFO Stopping model uploader_1 with job name uploader_1-CG4ZUWNOOWS3
11:46:14 osprey.us.cray.com SmartSim[126059] INFO Stopping model orchestrator_0 with job name orchestrator_0-CG4ZUHB3EWC8


In [7]:
exp = Experiment("launch_streaming_hvd", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 16)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer_hvd(exp, alloc, 1, 8)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

11:46:15 osprey.us.cray.com SmartSim[126059] INFO Working in previously created experiment
11:46:48 osprey.us.cray.com SmartSim[126059] INFO Working in previously created experiment
11:46:55 osprey.us.cray.com SmartSim[126059] INFO Working in previously created experiment
11:47:03 osprey.us.cray.com SmartSim[126059] INFO uploader_0(528401.9): New
11:47:03 osprey.us.cray.com SmartSim[126059] INFO uploader_1(528401.10): New
11:47:03 osprey.us.cray.com SmartSim[126059] INFO trainer(528401.11): New
11:47:08 osprey.us.cray.com SmartSim[126059] INFO uploader_0(528401.9): Running
11:47:08 osprey.us.cray.com SmartSim[126059] INFO uploader_1(528401.10): Running
11:47:08 osprey.us.cray.com SmartSim[126059] INFO trainer(528401.11): Running
11:47:13 osprey.us.cray.com SmartSim[126059] INFO uploader_0(528401.9): Running
11:47:13 osprey.us.cray.com SmartSim[126059] INFO uploader_1(528401.10): Running
11:47:13 osprey.us.cray.com SmartSim[126059] INFO trainer(528401.11): Running
11:47:18 osprey.us.cra

KeyboardInterrupt: 

In [8]:
slurm.release_allocation(alloc)

11:51:11 osprey.us.cray.com SmartSim[126059] INFO Releasing allocation: 528401
11:51:11 osprey.us.cray.com SmartSim[126059] INFO Successfully freed allocation 528401
11:51:20 osprey.us.cray.com SmartSim[126059] INFO uploader_0(528401.9): Cancelled
11:51:20 osprey.us.cray.com SmartSim[126059] INFO uploader_1(528401.10): Cancelled
11:51:20 osprey.us.cray.com SmartSim[126059] INFO trainer(528401.11): Cancelled
11:51:20 osprey.us.cray.com SmartSim[126059] INFO orchestrator_0(528401.8): Cancelled
